# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
#    file_path_list = glob.glob(os.path.join(root,'*'))
#    print(file_path_list)
     files_path_list = glob.glob('/home/workspace/event_data/*.csv')


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiate an empty list of rows that will be generated from each file
full_data_rows_list = []
    
# for every filepath in the file path list 
for f in files_path_list:

# read in the csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extract each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# create a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


# Part II. Implement Apache Cassandra Database tables. 

## Use the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# make a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# establish a connection and begin executing queries using a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [8]:
# set file name which holds the subset of columns and will act as the input file to be 
# loaded into the target tables
file = 'event_datafile_new.csv'                    

## Create tables to ask the following three questions of the data


### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [9]:
## Build table for Query 1 which gets the artist and song details heard during a particular session:
## Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_by_session_id"
query = query + """(session_id int, item_in_session int, artist text, song text, length float, 
                    PRIMARY KEY (session_id,item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)




In [10]:
## Load the data for Query 1 

# read the input csv file and create and execute the required INSERT statements
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_by_session_id(session_id,item_in_session,artist,song,length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Add in the SELECT statement for Query 1 to verify the data was entered into the table
query = "select artist,song,length from music_by_session_id where session_id = 338 and item_in_session  = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [12]:
## Build the table for Query 2 which gets the artist, song and user details by user and session: 
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS music_by_user_id"
query = query + """(user_id int,session_id int, item_in_Session int,artist text, song text, 
                    first_name text, last_name text, PRIMARY KEY ((user_id,session_id),item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    
    
                    

In [13]:
## Load the data for Query 2

# read the input csv file for Query 2 and create and execute the required INSERT statements
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_by_user_id(user_id,session_id,item_in_session,artist,song,first_name,last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0], line[9],line[1],line[4]))    

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## Add in the SELECT statement for Query 2 to verify the data was entered into the table
query = "select artist,song,first_name,last_name from music_by_user_id where user_id= 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [15]:
## Build the table for Query 3 which gets the user details for a particular song: 
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS music_by_song"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                        


In [16]:
## Load the data for Query 3

# read the input csv file for Query 3 and create and execute the required INSERT statements
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_by_song(song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))    



#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
## Add in the SELECT statement for Query 3 to verify the data was entered into the table
query = "select first_name,last_name from music_by_song where song= 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)
                    


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions
table_list = ["music_by_session_id", "music_by_user_id", "music_by_song"]
for table in table_list:
    query = "DROP TABLE " + table
    try:
        session.execute(query)
    except Exception as e:
        print(e)

    

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()